In [1]:
import pickle
import os

with open(r".\BL_Snipets\Stanford\Stanford_Nova_Regular_Ref_100.pkl", "rb") as f:
    data = pickle.load(f)

In [10]:
for key, value in data.items():
    print(f"{key}: {str(value)[:100]}")


cell_id: Stanford_Nova_Regular_Ref_100
cycle_data: [{'cycle_number': 1, 'current_in_A': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.
form_factor: pouch
anode_material: graphite
cathode_material: LiNi0.5Mn0.3Co0.2O2
electrolyte_material: None
nominal_capacity_in_Ah: 0.24
depth_of_charge: 1.0
depth_of_discharge: 1.0
already_spent_cycles: 0
max_voltage_limit_in_V: 4.4
min_voltage_limit_in_V: 3
max_current_limit_in_A: None
min_current_limit_in_A: None
reference: None
description: None
charge_protocol: [{'rate_in_C': 1.0, 'current_in_A': None, 'voltage_in_V': None, 'power_in_W': None, 'start_voltage_i
discharge_protocol: [{'rate_in_C': 0.75, 'current_in_A': None, 'voltage_in_V': None, 'power_in_W': None, 'start_voltage_
SOC_interval: [0, 1]


In [17]:
type(data["discharge_protocol"][0])
len(data["cycle_data"][0])
print([x for x in data["cycle_data"][0].keys()])


['cycle_number', 'current_in_A', 'voltage_in_V', 'charge_capacity_in_Ah', 'discharge_capacity_in_Ah', 'time_in_s', 'temperature_in_C', 'internal_resistance_in_ohm']
